## Agentic Workflow Design Pattern - Prompt Chaining

Prompt Chaining is a foundational design pattern in agentic AI workflows. It breaks down complex tasks into a sequence of smaller, interconnected prompts, where the output of one large language model (LLM) call becomes the input for the next.

This approach enables structured, step-by-step reasoning and improves reliability, controllability, and performance compared to handling everything in a single large prompt.

### How It Works
In prompt chaining:

1. A complex goal decomposes into logical subtasks.
2. Each subtask is handled by a specialized prompt.
3. Outputs are passed sequentially (often formatted as structured data like JSON for easy parsing).
4. The chain continues until the final result is produced.

This mimics a pipeline or assembly line, allowing focused processing at each stage. It evolved from techniques like Chain-of-Thought prompting but emphasizes modular, sequential execution.

In [22]:
from typing_extensions import TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import END, StateGraph, START
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [23]:
class SharedState(TypedDict):
    query: str
    model: ChatGroq
    from_ml_topic: bool
    ai_answer: str

class GraderOutput(TypedDict):
    from_machine_learning_topic: bool

In [24]:
def build_model(shared_state: SharedState):
    model = ChatGroq(model_name="llama-3.3-70b-versatile")
    shared_state['model'] = model
    return shared_state

In [25]:
def get_query_topic(shared_state: SharedState):
    print("Determining if the query is related to machine learning topics...")
    
    prompt = """
        You are a classifier that determines if a user's query is related to machine learning topics.
        Given the user's query, return True if it is related to machine learning topics, otherwise return False.
    """

    model = shared_state['model']
    structured_llm_grader = model.with_structured_output(GraderOutput)
    query = shared_state['query']

    grade_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            ("human", "User's Query: \n\n {query}"),
        ]
    )

    retrieval_grader = grade_prompt | structured_llm_grader

    result = retrieval_grader.invoke({"query": query})
    shared_state['from_ml_topic'] = result['from_machine_learning_topic']

    return shared_state

In [26]:
def grader_node(shared_state: SharedState):
    if shared_state['from_ml_topic']:
        return "continue"

    return "exit"

In [27]:
def answer_query(shared_state: SharedState):
    print("Answering the user's query...")
    prompt = """
        You are an expert in machine learning. Answer the user's query under 200 words.
    """
    model = shared_state['model']
    query = shared_state['query']
    answer_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            ("human", "User's Query: \n\n {query}"),
        ]
    )
    answer_chain = answer_prompt | model | StrOutputParser()
    result = answer_chain.invoke({"query": query})
    shared_state['ai_answer'] = result

    return shared_state

In [28]:
def build_graph():
    workflow = StateGraph(SharedState)

    # Add Nodes
    workflow.add_node(build_model, "build_model")
    workflow.add_node(get_query_topic, "get_query_topic")
    workflow.add_node(answer_query, "answer_query")

    workflow.add_edge(START, "build_model")
    workflow.add_edge("build_model", "get_query_topic")
    workflow.add_conditional_edges(
        "get_query_topic", 
        grader_node, 
        { 
            "continue": "answer_query",
            "exit": END 
        }
    )
    workflow.add_edge("answer_query", END)

    return workflow.compile()

In [33]:
# Query 1: "What are the latest advancements in machine learning?"
# Query 2: "What is the capital of Sri lanka?"
def execute_prompt_chain_workflow():
    workflow = build_graph()
    initial_state: SharedState = {
        "query": "What is the capital of Sri lanka?",
    }

    agent_response = workflow.invoke(initial_state)
    print(agent_response)

    if agent_response['from_ml_topic']:
        print("AI's Answer:", agent_response['ai_answer'])
    else:
        print("The query is not related to machine learning topics.")

In [34]:
load_dotenv()
execute_prompt_chain_workflow()

Determining if the query is related to machine learning topics...
{'query': 'What is the capital of Sri lanka?', 'model': ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000204968D8A10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000204968D8EC0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********')), 'from_ml_topic': False}
The query is not related to machine learning topics.
